In [1]:
%pip install pandas dash jupyter-dash


   -------------------- ------------------- 1/2 [jupyter-dash]
   ---------------------------------------- 2/2 [jupyter-dash]

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# 1. Load Data directly from the URL to ensure it works
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(URL)

# 2. Initialize the app using the standard Dash class
# In modern Dash, this automatically detects the Jupyter environment
app = dash.Dash(__name__)

# 3. Layout (TASK 1 & 3)
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True),
    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]),

    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# 4. Callbacks (TASK 2 & 4)
@app.callback(Output('success-pie-chart', 'figure'),
              Input('site-dropdown', 'value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_counts = filtered_df.groupby('class').size().reset_index(name='counts')
        fig = px.pie(site_counts, values='counts', names='class', title=f'Success Rate for {entered_site}')
    return fig

@app.callback(Output('success-payload-scatter-chart', 'figure'),
              [Input('site-dropdown', 'value'), Input('payload-slider', 'value')])
def update_scatter(entered_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    df_filtered = spacex_df[mask]
    if entered_site == 'ALL':
        fig = px.scatter(df_filtered, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    else:
        df_site = df_filtered[df_filtered['Launch Site'] == entered_site]
        fig = px.scatter(df_site, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return fig

# 5. NEW Jupyter Running Logic
# Instead of run_server, use app.run with notebook=True
if __name__ == '__main__':
    app.run(jupyter_mode='inline', port=8050)